## Setting up the Google Earth Engine Project

First we need to create a Google Project with Google Earth Engine enabled, this requires a Google Account.  
You can create a free, personal Google Account by visiting <https://earthengine.google.com/>, clicking on 'Get Started' and following the instructions.  
After you have logged in you will be greeted with Get Started Using Earth Engine screen:  

![Create a Project](TutorialImages/createProject.png)

Click on Register a Noncommercial or Commercial Cloud Project button and follow the instructions.  
Take note of the *project ID* and *project name* during the creation of the project.


# Installing the Google Earth Engine API and importing it in python

The next step is to make sure we have the earthengine API installed.  

If you are using pip you can issue the following command in a terminal:  
    pip install earthengine-api --upgrade  

In this notebook we also use the following additional packages:  
geemap <https://geemap.org/>  
    pip install geemap

In [ ]:
# Import the ee API
import ee
# Authenticate using ee
# This command will open a new window asking you to sign in with your Google Account
# Then you will have to enable permissions for the notebook to access your Google Account
# The authentication flow will generate a token which you will need to provide in a pop-up box.
# The token will be saved and valid for a week.
ee.Authenticate()

In [ ]:
# Initialize your project, using the project ID you took note of during the creation of your Google Earth Engine Project
projectID = 'ee-hkvdeveloper'
ee.Initialize(project=projectID)

In [ ]:
# Now that you are authenticated and have initialized your project we are able to use Google Earth Engine API.
# For example we can get precipiation from the CHRIPS database using coordinates and display this on a geemap
import geemap

# The following function creates a bounding box (Region of Interest) based on given Latitude/Longitude
# This is handy so we can limit the amount of data we retrieve and display

def create_bounding_box(center_lat, center_lon, width, height):
    # Calculate half-width and half-height
    half_width = width / 2
    half_height = height / 2

    # Define the bounding box coordinates
    bounding_box = [
        [center_lon - half_width, center_lat - half_height],  # Lower-left corner
        [center_lon - half_width, center_lat + half_height],  # Upper-left corner
        [center_lon + half_width, center_lat + half_height],  # Upper-right corner
        [center_lon + half_width, center_lat - half_height],  # Lower-right corner
        [center_lon - half_width, center_lat - half_height],  # Closing the polygon
    ]

    return ee.Geometry.Polygon(bounding_box, None, False)

# Example: Create a bounding box around a point (e.g., center at [0, 0]) with width and height (based on degrees)
center_latitude = -18.665695
center_longitude = 35.529564
width = 5
height = 5

# Create the bounding box
roi = create_bounding_box(center_latitude, center_longitude, width, height)

# Define the time period
start_date = '2022-01-01'
end_date = '2022-01-02'

# Load CHRIPS precipitation dataset
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate(ee.Date(start_date), ee.Date(end_date)).select('precipitation')

# Clip the dataset to the region of interest
chirps_roi = chirps.map(lambda img: img.clip(roi))

# Create a Map and add the precipitation layer
Map = geemap.Map()
# Center the map based on the region of interest
Map.centerObject(roi, 10)
# Add the CHIRPS precipitation layer and assign a style
Map.addLayer(chirps_roi, {
    'min': 0,
    'max': 20,
    'opacity': 0.5,
    'palette': ['white', 'blue']
}, 'Mean Precipitation')
Map.addLayerControl()
Map

In [ ]:
# Define the time period
start_date = '2022-01-01'
end_date = '2022-01-02'

# Define Earth Engine datasets used
BASINS_ID = 'WWF/HydroSHEDS/v1/Basins/hybas_6'
BOUNDARIES_ID = 'FAO/GAUL/2015/level1'
CLIMATE_ID = 'UCSB-CHG/CHIRPS/DAILY'

# Retrieve the basins
basins = ee.FeatureCollection(BASINS_ID)

# Filter the boundaries to include only madagascar
madagascarBoundaries = ee.FeatureCollection(BOUNDARIES_ID).filter(
    'ADM0_NAME == "Madagascar"'
)

# Filter the basins to include only the basins inside the boundaries of madagascar
madagascar_basins = basins.filterBounds(madagascarBoundaries)

# Load the CHIRPS dataset
precipitation = ee.ImageCollection(CLIMATE_ID).filterDate(ee.Date(start_date), ee.Date(end_date)).select('precipitation')

# Convert the precipitation image collection to a single multi-band image containing all of the bands of every image in the collection
precipitation_bands = ee.Image(precipitation.toBands())

# Apply a reducer over the area of each feature in the given collection.
madagascar_basins = precipitation_bands.reduceRegions(
    collection=madagascar_basins,
    reducer=ee.Reducer.sum()
)

# Convert the reduced basins to a pandas dataframe
madagascar_basins_dataframe = ee.data.computeFeatures({
    'expression': madagascar_basins,
    'fileFormat': 'PANDAS_DATAFRAME'
})

madagascar_basins_dataframe